# Explore NUTS information

In [ ]:
# run if anytree or geopandas not installed
%pip install anytree geopandas

In [ ]:
import geopandas as gpd
from anytree import Node, RenderTree, PreOrderIter

In [ ]:
nuts_file = "../data/in/NUTS_RG_20M_2024_4326.shp.zip"

In [ ]:
nuts = gpd.read_file(nuts_file)

In [ ]:
# build tree structure from NUTS_ID
dummy_root = Node("WORLD", parent=None, level=-1)

nuts_ids = nuts.NUTS_ID.unique()
nuts_ids = sorted(
    nuts_ids
)  # sort to ensure parent nodes are created before child nodes

nodes = {"WORLD": dummy_root}

for nuts_id in nuts_ids:
    level = len(nuts_id) - 2
    parent_code = nuts_id[:-1] if level > 0 else None

    if parent_code is None:
        # country level will have dummy root as parent
        nodes[nuts_id] = Node(nuts_id, parent=dummy_root, level=level)
    else:
        nodes[nuts_id] = Node(nuts_id, parent=nodes[parent_code], level=level)

In [ ]:
# double check, there should be 1798 nodes in total, excluding the dummy root
node_count = sum(1 for node in PreOrderIter(dummy_root)) - 1  # exclude dummy root
assert node_count == 1798

In [ ]:
# count for leaf nodes
leaf_count = sum(1 for node in PreOrderIter(dummy_root) if node.is_leaf)
leaf_count

In [ ]:
# render the tree from a specific node
for pre, _, node in RenderTree(nodes["BA"]):
    print(f"{pre}{node.name} (level: {node.level})")

In [ ]:
# check if there are nodes with level > 3
nuts_gt3_nodes = []
for node in PreOrderIter(dummy_root):
    if node.level > 3:
        nuts_gt3_nodes.append(node.name)
nuts_gt3_nodes

In [ ]:
# leaf nodes that are not at NUTS3 level
not_nuts3_leaf_nodes = [
    node.name for node in PreOrderIter(dummy_root) if node.is_leaf and node.level < 3
]
not_nuts3_leaf_nodes